In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# **Heart Failure Prediction**

"*Cardiovascular diseases(CVDs) are the number 1 cause of death globally, taking an estimated 17.9 million lives each year. CVDs are group of disorders of the heart and blood vessels include coronary heart disease, cerebrovascular disease, rheumatic heart disease and other conditions.*"

--- [World Health Organization](https://www.who.int/health-topics/cardiovascular-diseases#tab=tab_1)

According to World Health Organization, four out of 5 CVD deaths are due to **heart attacks and srokes**, and one third of these occur prematurely in people under 70 years of age.

The dataset of my project for [Data Analytics with Python: Zero to Pandas](https://jovian.ml/learn/data-analysis-with-python-zero-to-pandas) is Heart **Failure Prediction** which I obtained from the [Kaggle - Heart Failure Prediction](https://www.kaggle.com/andrewmvd/heart-failure-clinical-data).

This dataset contains 12 features, 299 rows that can be used to predict mortality by heart failure.

Detailed explanation of each feature:

* **age**
* **anaemia** → decrease of red blood cells or hemoglobin (boolean)

> * No : 0
> * Yes : 1

**creatinine_phosphokinase** → Level of the CPK enzyme in the blood (mcg/L)

**diabetes** → if the patient has diabetes (boolean)

> * No: 0
> * Yes : 1

**enjection_fraction** → Percentage of blood leaving the heart ar each contraction (percentage)

**high_blood_pressure** → if the patient has hypertension ( boolean)
> * No : 0
> * Yes : 1

**platelets** → Platelets in the blood (kiloplateletes/mL)

**serum_creatinine** → Level of serum creatinine in the blood (mg/dL)

**serum_sodium** → Level of serum sodium in the blood (mEq/L)

**sex** → Woman or man (binary)
> * Gender of patient Male = 1
> * Gender of patient Female = 0

**smoking** → if the patient smokes or not (boolean)
> * No : 0
> * Yes : 1

**time** → follow-up period (days)

**TARGET:**

> > **death_event** → if the patient deceased during the follow-up period (boolean)
> > * No : 0
> > * Yes : 1

#  **Data Preparation and Cleaning**

In this part, I will load the dataset into a data frame without any changes. In addition to this, I will explore the number of rows and columns, observe the missing data and find a solution to **handle missing data or invalid data**.

In [ ]:
#Import necessary libraries

import pandas as pd
import numpy as np
import seaborn as sns
from plotly.offline import plot,iplot,init_notebook_mode

import plotly.graph_objects as go
# The plotly.graph_objects module (typically imported as go ) contains an automatically-generated hierarchy of 
# Python classes which represent non-leaf nodes in this figure schema. 
# The term "graph objects" refers to instances of these classes.
# https://plotly.com/python/graph-objects/#:~:text=machine%2Dreadable%20form.-,The%20plotly.,to%20instances%20of%20these%20classes.


import plotly.express as px
# The plotly.express module (usually imported as px ) contains functions that can create entire figures at once, 
# and is referred to as Plotly Express or PX. Plotly Express is a built-in part of the plotly library, 
# and is the recommended starting point for creating most common figures.
# https://plotly.com/python/plotly-express/#:~:text=Dash%20Enterprise%20Overview-,Overview,for%20creating%20most%20common%20figures.


import warnings

import lightgbm as lgb
# LightGBM can use categorical features as input directly. 
# It doesn't need to convert to one-hot encoding, and is much faster than one-hot encoding (about 8x speed-up). 
# Note: You should convert your categorical features to int type before you construct Dataset .
# https://lightgbm.readthedocs.io/en/latest/Python-Intro.html
# https://medium.com/@pushkarmandot/https-medium-com-pushkarmandot-what-is-lightgbm-how-to-implement-it-how-to-fine-tune-the-parameters-60347819b7fc
# LightGBM, short for Light Gradient Boosting Machine, is a free and open source distributed gradient boosting 
# framework for machine learning originally developed by Microsoft.[4][5] It is based on decision tree algorithms 
# and used for ranking, classification and other machine learning tasks. The development focus 
# is on performance and scalability.
# https://en.wikipedia.org/wiki/LightGBM

from IPython.display import HTML 

init_notebook_mode(connected=True)
warnings.filterwarnings("ignore")

In [ ]:
heart_failure_raw_df = pd.read_csv("../input/heart-failure-clinical-data/heart_failure_clinical_records_dataset.csv")
heart_failure_raw_df

The dataset contains 288 responses to 13 features.

Let's view the list of columns in the data frame

In [ ]:
heart_failure_raw_df.columns

In [ ]:
selected_columns = heart_failure_raw_df.columns
len(selected_columns)

Let's extract a copy of the data from these columns into a new data frame **heart_failure_df**, which I can continue to modify further without affection the original data frame.

In [ ]:
heart_failure_df = heart_failure_raw_df[selected_columns].copy()
heart_failure_df

In [ ]:
heart_failure_df.shape

In [ ]:
heart_failure_df.info()

In [ ]:
heart_failure_df.isnull()

In [ ]:
heart_failure_df.isnull().sum()

All columns except for age, platelets,serum_creatinine have the data type integer. So, I don't have to convert object values to numeric values.

According to all columns, they don't contain empty values (missing data). So, I don't do anything to handle the missing data.

# **Exploratory Data Analysis (EDA) and Visualization**

Before asking interesting questions, I should understand **which column is so important for hearth failure prediction**. By exploring these variables in order to understand the dataset, I might get the best prediction.

Let's begin by importing matplotlib.pyplot and seaborn

In [ ]:
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

sns.set_style('darkgrid')

# Matplotlib uses matplotlibrc configuration files to customize all kinds of properties, 
# which we call 'rc settings' or 'rc parameters'
# https://matplotlib.org/stable/tutorials/introductory/customizing.html

matplotlib.rcParams['font.size'] = 14
matplotlib.rcParams['figure.figsize'] = (9, 5)
matplotlib.rcParams['figure.facecolor'] = '#00000000'

# Visualization of Target Column

In [ ]:
target = heart_failure_df["DEATH_EVENT"]
sns.countplot(heart_failure_df.DEATH_EVENT)

In [ ]:
counts = heart_failure_df.DEATH_EVENT.value_counts()
counts

In [ ]:
# Getting the count and percentage column by using target column

counts = heart_failure_df.DEATH_EVENT.value_counts()
percentage = heart_failure_df.DEATH_EVENT.value_counts(normalize=True).mul(100).round(1).astype(str) + '%'
pd.DataFrame({'Counts':counts,'Percentage': percentage})

# With normalize set to True , returns the relative frequency by dividing all values by the sum of values. 
# Bins can be useful for going from a continuous variable to a categorical variable; 
# instead of counting unique apparitions of values, divide the index in the specified number of half-open bins.

In [ ]:
y = heart_failure_df.loc[:,"DEATH_EVENT"]
y

In [ ]:
y_dropped = heart_failure_df.drop("DEATH_EVENT",axis = 1)
y_dropped

In [ ]:
X = y_dropped.loc[:,:]
# X = y_dropped
X

In [ ]:
from sklearn.model_selection import train_test_split

# Removing the target column from real dataset
y = heart_failure_df.loc[:,"DEATH_EVENT"]
y_dropped = heart_failure_df.drop("DEATH_EVENT",axis = 1)
X = y_dropped.loc[:,:]

# Data Splitting to determine Feature Importance 
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size= 0.30, random_state = 42)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

# **Feature Importance without Gain**

Without Gain refers the importance of the features according to the branching of the tree.


In [ ]:
import lightgbm as lgb

# Indicating all specific params
fit_params={"early_stopping_rounds":10, 
            "eval_metric" : 'auc', 
            "eval_set" : [(X_test,y_test)],
            'eval_names': ['valid'],
            'verbose': 100,
            'feature_name': 'auto', # that's actually the default
            'categorical_feature': 'auto' # that's actually the default
           }

# n_estimators is set to a "large value". The actual number of trees build will depend on early stopping and 1000 define only the absolute maximum
clf = lgb.LGBMClassifier(num_leaves= 15, max_depth=-1, 
                         random_state=314, 
                         silent=True, 
                         metric='None', 
                         n_jobs=4, 
                         n_estimators=1000,
                         colsample_bytree=0.9,
                         subsample=0.9,
                         learning_rate=0.1)

# force larger number of max trees and smaller learning rate
clf.fit(X_train, y_train, **fit_params)

feat_imp = pd.Series(clf.feature_importances_, index=X.columns)
feat_imp.nlargest(30).plot(kind='barh', figsize=(8,10))

# **Feature Importance with Gain**

With Gain refers the improtance of the features according to the gain in each brach.

In [ ]:
class LGBMClassifier_GainFE(lgb.LGBMClassifier):
    @property 
    # We used @property decorator to give "special" functionality such as deleters to certain methods
    def feature_importances_(self):
        if self._n_features is None:
            raise LGBMNotFittedError('No feature_importances found. Need to call fit beforehand.')
        return self.booster_.feature_importance(importance_type='gain')

clf2 = LGBMClassifier_GainFE(num_leaves= 15, max_depth=-1, 
                         random_state=314, 
                         silent=True, 
                         metric='None', 
                         n_jobs=4, 
                         n_estimators=1000,
                         colsample_bytree=0.9,
                         subsample=0.9,
                         learning_rate=0.1) 

clf2.fit(X_train, y_train, **fit_params)

feat_imp = pd.Series(clf2.feature_importances_, index=X.columns)
feat_imp.nlargest(30).plot(kind='barh', figsize=(8,10))


When the 2nd graphic is examined, the features that affect the target column most were determined.

So, I will select only 5 features: time, serum_creatinine, ejection_fraction, age, creatinine_phosphokinase.

# **Analysis of Important Features**

**TIME**

In [ ]:
%matplotlib inline
fig = px.histogram(heart_failure_df, x="time", color="DEATH_EVENT", marginal="violin", hover_data=heart_failure_df.columns, 
                   title ="Distribution of TIME Vs DEATH_EVENT", 
                   labels={"time": "TIME"})
fig.show()

**If the follow-up time of a person's heart disease increases, the mortality rate of the heart disease decreases significantly.**

**SERUM CREATININE**

In [ ]:
fig = px.histogram(heart_failure_df, x="serum_creatinine", color="DEATH_EVENT", marginal="violin", hover_data=heart_failure_df.columns, 
                   title ="Distribution of SERUM CREATININE Vs DEATH_EVENT", 
                   labels={"serum_creatinine": "SERUM CREATININE"})
fig.show()

When the level of serum creatinine in the blood is between 0 and 2 mg/dL, it is observed that the patient's survival rates is higher than the death rate.

**EJECTION FRACTION**

In [ ]:
fig = px.histogram(heart_failure_df, x="ejection_fraction", color="DEATH_EVENT", marginal="violin", hover_data=heart_failure_df.columns, 
                   title ="Distribution of EJECTION FRACTION Vs DEATH_EVENT", 
                   labels={"ejection_fraction": "EJECTION FRACTION"})
fig.show()

**AGE**

In [ ]:
fig = px.histogram(heart_failure_df, x="age", color="DEATH_EVENT", marginal="violin", hover_data=heart_failure_df.columns, 
                   title ="Distribution of AGE Vs DEATH_EVENT", 
                   labels={"age": "AGE"})
fig.show()


A person between the ages of 40 and 70 is more likely to survive.

**CREATININE PHOSPHOKINASE**

In [ ]:
fig = px.histogram(heart_failure_df, x="creatinine_phosphokinase", color="DEATH_EVENT", marginal="violin", hover_data=heart_failure_df.columns, 
                   title ="Distribution of CREATININE PHOSPHOKINASE Vs DEATH_EVENT", 
                   labels={"creatinine_phosphokinase": "CREATININE PHOSPHOKINASE"})
fig.show()

In [ ]:
heart_failure_df

In [ ]:
heart_failure_df[["sex","diabetes","smoking","anaemia","high_blood_pressure","DEATH_EVENT"]]

In [ ]:
heart_failure_df.groupby(["sex","diabetes","smoking","anaemia","high_blood_pressure","DEATH_EVENT"])

In [ ]:
heart_failure_df.groupby(["sex","diabetes","smoking","anaemia","high_blood_pressure","DEATH_EVENT"])["age"]

In [ ]:
heart_failure_df.groupby(["sex","diabetes","smoking","anaemia","high_blood_pressure","DEATH_EVENT"])["age"].count()

In [ ]:
sun = heart_failure_df.groupby(["sex","diabetes","smoking","anaemia","high_blood_pressure","DEATH_EVENT"])["age"].count().reset_index()
sun.columns= ["sex","diabetes","smoking","anaemia","high_blood_pressure","DEATH_EVENT", "count"]
sun

In [ ]:
sun.loc[sun["sex"]== 0 , "sex"] = "female"
sun.loc[sun["sex"]== 1, "sex"] = "male"

sun.loc[sun["diabetes"]== 0 , "diabetes"] = "no diabetes"
sun.loc[sun["diabetes"]== 1, "diabetes"] = "diabetes"

sun.loc[sun['DEATH_EVENT'] == 0,'DEATH_EVENT'] = "LIVE"
sun.loc[sun['DEATH_EVENT'] == 1, 'DEATH_EVENT'] = 'DEATH'

fig = px.sunburst(sun, 
                  path=["sex","diabetes","DEATH_EVENT"],
                  values="count",
                  title="Gender & Diabetes Sunburst Chart ",
                  width=600,
                  height=600)

fig.show()

Whether the patient are diabetic or not, the survival rate is higher than the death rate.



In [ ]:
sun.loc[sun["smoking"]== 0 , "smoking"] = "non smoking"
sun.loc[sun["smoking"]== 1, "smoking"] = "smoking"

fig = px.sunburst(sun, 
                  path=["sex","smoking","DEATH_EVENT"],
                  values="count",
                  title="Gender & Smoking Sunburst Chart ",
                  width=600,
                  height=600)

fig.show()

**Whether a patient is a smoker or not, the survival rate is higher than the death rate. However, that does not mean it carries no risk.**

In [ ]:
sun.loc[sun["anaemia"]== 0 , "anaemia"] = "no anaemia"
sun.loc[sun["anaemia"]== 1, "anaemia"] = "anaemia"

fig = px.sunburst(sun, 
                  path=["sex","anaemia","DEATH_EVENT"],
                  values="count",
                  title="Gender & Anaemia  Sunburst Chart ",
                  width=600,
                  height=600)

fig.show()

**Whether the patient has anaemia or not, the survival rate is higher than the death rate.**

In [ ]:
sun.loc[sun["high_blood_pressure"]== 0 , "high_blood_pressure"] = "no high_blood_pressure"
sun.loc[sun["high_blood_pressure"]== 1, "high_blood_pressure"] = "high_blood_pressure"

fig = px.sunburst(sun, 
                  path=["sex","high_blood_pressure","DEATH_EVENT"],
                  values="count",
                  title="Gender & High Blood Pressure Sunburst Chart ",
                  width=600,
                  height=600)

fig.show()

Whether the patient has high blood pressure or not, the survival rate is higher than the death rate.

Let us save and upload our work to Jovian before continuing

# **Asking and Answering Questions**

So far I have observed how different features are related together. I will now look for answers by asking questions about how these features together affect the deadth event.

**Q1: How does age and gender difference affect the mortality rate of the heart disase?**

In [ ]:
fig = px.violin(heart_failure_df, x="age",y="sex", color="DEATH_EVENT",
                box=True,points="all",hover_data=heart_failure_df.columns,
                title="Analysis of Gender and Age on Survival People",
                orientation="h")

fig.show()


Gender : Male = 1, Female = 0

DEATH_EVENT : Death = 1, Live = 0

------------------------------------

When I look at the violin graph, the patients have a higher chance of survival between the ages of 40 and 70.

The survival rate is high for both male and female between 50 to 65.

**Q2: How does the person's smoking or non-smoking, as well as age and gender difference affect the mortality rate of the heart disase?**

In [ ]:
fig = px.violin(heart_failure_df, x="age",y="smoking", color="DEATH_EVENT",
                box=True,points="all",hover_data=heart_failure_df.columns,
                title="Analysis of Age and Smoking on Survival People",
                orientation="h")
fig.show()

Smoking : Smoking = 1, Non Smoking = 0

DEATH_EVENT : Death = 1, Live = 0

--------------------------------------

When I look at the violin graph, the patients who are non smoking have a higher chance of survival between the ages of 50 and 65. On the other hand, the patients who are smoking have change of survival between the ages 50 to 60.

Death event for smoking person is higher than non smoking person.

In [ ]:
heart_failure_df["sex"][heart_failure_df["smoking"]==1]

In [ ]:
heart_failure_df["DEATH_EVENT"][heart_failure_df["smoking"]==1]

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Violin(x=heart_failure_df["sex"][heart_failure_df["smoking"]==1],
                        y=heart_failure_df["DEATH_EVENT"][heart_failure_df["smoking"]==1],
                        legendgroup=1, scalegroup=1,name="Death",
                        side="negative",line_color="blue"))

fig.add_trace(go.Violin(x=heart_failure_df["sex"][heart_failure_df["smoking"]==0],
                        y=heart_failure_df["DEATH_EVENT"][heart_failure_df["smoking"]==0],
                        legendgroup=0, scalegroup=0,name="Live",
                        side="positive",line_color="pink"))



fig.update_traces(meanline_visible=True)
fig.update_layout(violingap=0,violinmode="overlay",
                  title= "Analysis of Gender & Smoking on Survival Person",
                  xaxis_title="Gender",
                  yaxis_title="Smoking",
                  legend_title="Death Event")
fig.show()

Gender : Male = 1, Female = 0

Smoking : Smoking = 1, Non Smoking = 0

DEATH_EVENT : Death = 1, Live = 0

-----------------------------

When I look at the violin graph, the mortality rate for male smoking person is higher than female smoking person. The mortality rate for male non smoking person is higher than female smoking person. The survival rate for both of them(female & male) are equal each other.

**Q3: How does the person's diabetes or not, as well as age and gender difference affect the mortality rate of the heart disase?**

In [ ]:
fig = px.violin(heart_failure_df, x="age",y="diabetes", color="DEATH_EVENT",
                box=True,points="all",hover_data=heart_failure_df.columns,
                title="Analysis of Age and Diabetes on Survival People",
                orientation="h")
fig.show()

Diabetes : Diabets = 1, No Diabetes = 0

DEATH_EVENT : Death = 1, Live = 0

----------------

When I look at the violin graph, the patients who are not diabetes have a higher chance of survival between the ages of 50 and 65. On the other hand, the patients who are diabetes have change of survival between the ages 50 to 65.

Death event for diabetes person is higher than no diabetes person.

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Violin(x=heart_failure_df["sex"][heart_failure_df["diabetes"]==1],
                        y=heart_failure_df["DEATH_EVENT"][heart_failure_df["diabetes"]==1],
                        legendgroup=1, scalegroup=1,name="Death",
                        side="negative",line_color="blue"))

fig.add_trace(go.Violin(x=heart_failure_df["sex"][heart_failure_df["diabetes"]==0],
                        y=heart_failure_df["DEATH_EVENT"][heart_failure_df["diabetes"]==0],
                        legendgroup=0, scalegroup=0,name="Live",
                        side="positive",line_color="pink"))



fig.update_traces(meanline_visible=True)
fig.update_layout(violingap=0,violinmode="overlay",
                  title= "Analysis of Gender & Diabetes on Survival Person",
                  xaxis_title="Gender",
                  yaxis_title="Diabetes",
                  legend_title="Death Event")
fig.show()

Diabetes : Diabetes = 1, No Diabetes = 0

Gender : Male = 1 , Female = 0

DEATH_EVENT : Death = 1, Live = 0

------

The survival and mortality rate for both of them(female & male) are approximately equal to each other. People who have diabetes don't affect the mortlality rate as much as people who don't have diabetes.

**Q4: How does the person's anaemia or not, as well as age and gender difference affect the mortality rate of the heart disase?**

In [ ]:
fig = px.violin(heart_failure_df, x="age",y="anaemia", color="DEATH_EVENT",
                box=True,points="all",hover_data=heart_failure_df.columns,
                title="Analysis of Age and Anaemia on Survival People",
                orientation="h")
fig.show()

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Violin(x=heart_failure_df["sex"][heart_failure_df["anaemia"]==1],
                        y=heart_failure_df["DEATH_EVENT"][heart_failure_df["anaemia"]==1],
                        legendgroup=1, scalegroup=1,name="Death",
                        side="negative",line_color="blue"))

fig.add_trace(go.Violin(x=heart_failure_df["sex"][heart_failure_df["anaemia"]==0],
                        y=heart_failure_df["DEATH_EVENT"][heart_failure_df["anaemia"]==0],
                        legendgroup=0, scalegroup=0,name="Live",
                        side="positive",line_color="pink"))



fig.update_traces(meanline_visible=True)
fig.update_layout(violingap=0,violinmode="overlay",
                  title= "Analysis of Gender & Anaemia on Survival Person",
                  xaxis_title="Gender",
                  yaxis_title="Anaemia",
                  legend_title="Death Event")
fig.show()

**Q5: How does the person's high blood pressure or not, as well as age and gender difference affect the mortality rate of the heart disase?**

In [ ]:
fig = px.violin(heart_failure_df, x="age",y="high_blood_pressure", color="DEATH_EVENT",
                box=True,points="all",hover_data=heart_failure_df.columns,
                title="Analysis of Age and High Blood Pressure on Survival People",
                orientation="h")
fig.show()

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Violin(x=heart_failure_df["sex"][heart_failure_df["high_blood_pressure"]==1],
                        y=heart_failure_df["DEATH_EVENT"][heart_failure_df["high_blood_pressure"]==1],
                        legendgroup=1, scalegroup=1,name="Death",
                        side="negative",line_color="blue"))

fig.add_trace(go.Violin(x=heart_failure_df["sex"][heart_failure_df["high_blood_pressure"]==0],
                        y=heart_failure_df["DEATH_EVENT"][heart_failure_df["high_blood_pressure"]==0],
                        legendgroup=0, scalegroup=0,name="Live",
                        side="positive",line_color="pink"))



fig.update_traces(meanline_visible=True)
fig.update_layout(violingap=0,violinmode="overlay",
                  title= "Analysis of Gender & High Blood Pressur on Survival Person",
                  xaxis_title="Gender",
                  yaxis_title="High Blood Pressure",
                  legend_title="Death Event")
fig.show()

**Q6: Why do anaemia, high blood pressure, diabetes, gender and smoking affect death events so little?**

Before that, let's see the results table of the features that affect the death event most. Anywhere you see in blue refers to the survivor, which I found that the following features had a great impact on the death event

In [ ]:
sns.pairplot(heart_failure_df,hue="DEATH_EVENT",vars =["time","serum_creatinine","ejection_fraction","age","creatinine_phosphokinase","platelets","serum_sodium"])

In [ ]:
data_cm = heart_failure_df[["time","serum_creatinine","ejection_fraction","age","creatinine_phosphokinase","platelets","serum_sodium"]]
data_cm

In [ ]:
data_cm.corr()

In [ ]:
sns.heatmap(data_cm.corr(), annot = True, vmin=-1, vmax=1, center= 0, cmap= 'coolwarm')

In [ ]:
import plotly.graph_objects as go

anaemia = heart_failure_df[heart_failure_df["anaemia"]==1]
no_anaemia = heart_failure_df[heart_failure_df["anaemia"]==0]

fig = go.Figure(go.Pie(
    title="Why does anaemia affect death events so little?",
    values=[len(anaemia[heart_failure_df["DEATH_EVENT"]==1]),
           len(anaemia[heart_failure_df["DEATH_EVENT"]==0]),
           len(no_anaemia[heart_failure_df["DEATH_EVENT"]==1]),
           len(no_anaemia[heart_failure_df["DEATH_EVENT"]==0])],
    labels=["Anaemia + No Survived",
          "Anaemia + Survived",
          "No Anaemia + No Survived",
          "No Anaemia + Survived"],
    hovertemplate = "%{label}: <br>Rate: %{percent} </br> %{text}"
))

fig.show()

In [ ]:
import plotly.graph_objects as go

high_blood_pressure = heart_failure_df[heart_failure_df["high_blood_pressure"]==1]
no_high_blood_pressure = heart_failure_df[heart_failure_df["high_blood_pressure"]==0]

fig = go.Figure(go.Pie(
    title="Why does high blood pressure affect death events so little?",
    values=[len(high_blood_pressure[heart_failure_df["DEATH_EVENT"]==1]),
           len(high_blood_pressure[heart_failure_df["DEATH_EVENT"]==0]),
           len(no_high_blood_pressure[heart_failure_df["DEATH_EVENT"]==1]),
           len(no_high_blood_pressure[heart_failure_df["DEATH_EVENT"]==0])],
    labels=["High Blood Pressure + No Survived",
          "High Blood Pressure + Survived",
          "No High Blood Pressure + No Survived",
          "No High Blood Pressure + Survived"],
    hovertemplate = "%{label}: <br>Rate: %{percent} </br> %{text}"
))

fig.show()

In [ ]:
import plotly.graph_objects as go

diabetes = heart_failure_df[heart_failure_df["diabetes"]==1]
no_diabetes = heart_failure_df[heart_failure_df["diabetes"]==0]

fig = go.Figure(go.Pie(
    title="Why does diabetes affect death events so little?",
    values=[len(diabetes[heart_failure_df["DEATH_EVENT"]==1]),
           len(diabetes[heart_failure_df["DEATH_EVENT"]==0]),
           len(no_diabetes[heart_failure_df["DEATH_EVENT"]==1]),
           len(no_diabetes[heart_failure_df["DEATH_EVENT"]==0])],
    labels=["Diabetes + No Survived",
          "Diabetes + Survived",
          "No Diabetes + No Survived",
          "No Diabetes + Survived"],
    hovertemplate = "%{label}: <br>Rate: %{percent} </br> %{text}"
))

fig.show()

In [ ]:
import plotly.graph_objects as go

male = heart_failure_df[heart_failure_df["sex"]==1]
female = heart_failure_df[heart_failure_df["sex"]==0]

fig = go.Figure(go.Pie(
    title="Why does gender affect death events so little?",
    values=[len(male[heart_failure_df["DEATH_EVENT"]==1]),
           len(male[heart_failure_df["DEATH_EVENT"]==0]),
           len(female[heart_failure_df["DEATH_EVENT"]==1]),
           len(female[heart_failure_df["DEATH_EVENT"]==0])],
    labels=["Male + No Survived",
          "Male + Survived",
          "Female + No Survived",
          "Female + Survived"],
    hovertemplate = "%{label}: <br>Rate: %{percent} </br> %{text}"
))

fig.show()

In [ ]:
import plotly.graph_objects as go

smoking = heart_failure_df[heart_failure_df["smoking"]==1]
no_smoking = heart_failure_df[heart_failure_df["smoking"]==0]

fig = go.Figure(go.Pie(
    title="Why does smoking affect death events so little?", 
    values=[len(smoking[heart_failure_df["DEATH_EVENT"]==1]),
           len(smoking[heart_failure_df["DEATH_EVENT"]==0]),
           len(no_smoking[heart_failure_df["DEATH_EVENT"]==1]),
           len(no_smoking[heart_failure_df["DEATH_EVENT"]==0])],
    labels=["Smoking + No Survived",
          "Smoking + Survived",
          "No Smoking + No Survived",
          "No Smoking + Survived"],
    hovertemplate = "%{label}: <br>Rate: %{percent} </br> %{text}"
))

fig.show()

When I look at all the results, even if none of above features are present in the patient, it does not affect the death event because the patient's survival rate is higher.


# **Inferences and Conclusion**

* First I tried to understand datasets which feature have what properties. Some contain only boolean values while others contain continuous values. After that, I determined the feature importance by using Light GBM algorithm. According to this result, the important features are quite different what I expected. So, I examined the data analysis by using Plotly, seaborn library. End of the exploration, I realized that the feature importance histogram of this dataset matches the exploratory data analysis results.

* In the part of the question, I asked different questions by dividing two part which is questions about the most important and less important features of the dataset. I explained why these features are less important by drawing different graphs. Moreover, I examined these features relationship with age and gender. Then, I showed the final and best result why these features are less important for death events.

# **References and Future Work**

References:

* https://www.kaggle.com/andrewmvd/heart-failure-clinical-data
* https://www.kaggle.com/sanchitakarmakar/heart-failure-prediction-visualization
* https://www.kaggle.com/isaienkov/data-visualization-modeling-82-acc-75-f1
* https://www.kaggle.com/nayansakhiya/heart-fail-analysis-and-quick-prediction-96-rate
* https://plotly.com/python/hover-text-and-formatting/
* https://plotly.com/python/creating-and-updating-figures/
* https://plotly.com/python/figure-labels/
* https://stackoverflow.com/questions/59993512/plotly-sunburst-typeerror-sunburst-got-an-unexpected-keyword-argument-path
* https://www.who.int/health-topics/cardiovascular-diseases#tab=tab_1

Future Work

* At first glance, we think to research the conditions under which the most influential features can override other factors. For example, we dropped that high blood pressure is definitely a high correlation with the heart, but we realized that it is a feature that needs much less attention than the result here. we would like to research this.
* we would like to see what impact this could have across the country, not just on the patient basis. We would like to observe if different geographies change this result.